In [1]:
import pandas as pd

df = pd.read_csv('large_amounts_of_data.csv')

df = df[['Hero ID','Team','Order','Lane','MatchID']]

# Count the number of duplicates for each unique MatchID
duplicates_count = df['MatchID'].value_counts()

# Print the count of duplicates for each unique MatchID
print(duplicates_count.to_frame())

            count
MatchID          
7425367208    720
7425288381    720
7425165105    720
7425319964    720
7425127995    710
...           ...
6752616741     10
6752609194     10
6752603883     10
6918595795     10
5               1

[2166 rows x 1 columns]


In [32]:
import pandas as pd
df = pd.read_csv('large_amounts_of_data.csv')
# Keep only the first 10 consecutive rows of each match ID
df_cleaned = df.groupby('MatchID').head(10)

# Reset the index of the cleaned DataFrame
df_cleaned.reset_index(drop=True, inplace=True)

# Save the cleaned DataFrame to a CSV file
df_cleaned.to_csv('cleaned_data.csv', index=False)
clean = pd.read_csv('cleaned_data.csv')

real_clean= clean.groupby('MatchID').head(10)

# Reset the index of the cleaned DataFrame
real_clean.reset_index(drop=True, inplace=True)
real_clean.to_csv('cleaned_data.csv', index=False)

df_real = pd.read_csv("cleaned_data.csv")
df_real = df_real.iloc[:, :44]
df_real = df_real[df_real["MatchID"] != 5]
print(len(df_real['MatchID'].value_counts()))
# df_real.head(10)

10014


In [28]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Create a Sequential model
model = keras.Sequential()

# Add a 2D convolutional layer with 32 filters
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(10, 44, 1)))

# Add a max-pooling layer
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output from the previous layer
model.add(layers.Flatten())

# Add a dense (fully connected) layer with 64 units
model.add(layers.Dense(64, activation='relu'))

# Add the output layer with as many units as the number of lane classes
# Adjust the number of units according to your specific problem
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


2023-11-09 01:50:49.584716: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-09 01:50:49.798404: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-09 01:50:49.798444: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-09 01:50:49.798851: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-09 01:50:49.855901: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-09 01:50:49.867583: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 8, 42, 32)         320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 4, 21, 32)         0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 2688)              0         
                                                                 
 dense (Dense)               (None, 64)                172096    
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 173066 (676.04 KB)
Trainable params: 173066 (676.04 KB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
from sklearn.linear_model import LogisticRegression

class DotaLanePredictor(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(n_inputs, n_outputs)

        
    def forward(self, x):